In [6]:
import csv
import ast

class Player:
    nick: str
    elo: int

players = {}

with open("matches.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        players_str = row["a"]
        players_list = ast.literal_eval(players_str)

        players_str = row["b"]
        players_list.append(ast.literal_eval(players_str))

        print(players_list)


print(len(players_list))


['Filip Rusztowicz', 'kb316244', 'kondrix', 'rafał radosław malczyk', ['filip_pro', 'marta', 'suchix', 'topol']]
['Filip Rusztowicz', 'kb316244', 'kondrix', 'rafał radosław malczyk', ['filip_pro', 'marta', 'suchix', 'topol']]
['Filip Rusztowicz', 'kb316244', 'kondrix', 'rafał radosław malczyk', ['filip_pro', 'marta', 'suchix', 'topol']]
['Filip Rusztowicz', 'kb316244', 'rafał radosław malczyk', ['filip_pro', 'marta', 'piloniak']]
['filip_pro', 'marta', 'piloniak', ['Filip Rusztowicz', 'kb316244', 'rafał radosław malczyk']]
['Filip Rusztowicz', 'rafał radosław malczyk', ['filip_pro', 'piloniak']]
['Filip Rusztowicz', 'rafał radosław malczyk', ['filip_pro', 'piloniak']]
['karol', 'oskarito', ['remzak', 'suchix']]
['oskarito', 'remzak', ['karol', 'michal']]
['oskarito', 'suchix', ['karol', 'michal']]
['karol', 'michal', ['oskarito', 'suchix']]
['karol', 'michal', ['oskarito', 'suchix']]
['oskarito', 'suchix', ['karol', 'michal']]
['filip_pro', 'suchix', ['oskarito', 'remzak']]
['filip_pro